In [1]:
import pandas as pd
import numpy as np

In [2]:
paises = pd.read_csv("continentes.csv").filter(["alpha-3", "name", "region", "sub-region"])
ue = (
    "BEL", "BGR", "CZE", "DNK", "DEU", "EST", "IRL", "GRC", "ESP",
    "FRA", "HRV", "ITA", "CYP", "LVA", "LTU", "LUX", "HUN", "MLT",
    "NLD", "AUT", "POL", "PRT", "ROU", "SVN", "SVK", "FIN", "SWE"
)

In [7]:
df = pd.read_csv("../relatorios/tabela_ex6.csv", index_col = 0)#.query("industry_name == 'Aircraft and spacecraft'")

In [39]:
with open(f'../relatorios/gl/gl.tex', 'w') as tf:
    tf.write(
        pd.concat([
            #Calculo para o mundo
            (df.dropna()
                .merge(paises, left_on = "exporter_iso3", right_on = "alpha-3", how = "left")
                .query("year % 2 == 0")
                .groupby(["year", "sub-region"]).GL_ij.mean().reset_index()
                .pivot_table(index = "sub-region", columns = "year", values = "GL_ij")
                .reset_index()
                .rename({"sub-region": "Sub-Região"}, axis = 1)
                .rename_axis("", axis = 1)
            ),
            #Cálculo para a união europeia
            (df
                .assign(ue = lambda _: _.exporter_iso3.isin(ue))
                .query("year % 2 == 0")
                .groupby(["ue", "year"]).GL_ij.mean().reset_index()
                .pivot_table(index = "ue", columns = "year", values = "GL_ij").reset_index()
                .query("ue == 1")
                .rename({"ue": "Sub-Região"}, axis = 1)
                .rename_axis("", axis = 1)
                .replace({True: "União Europeia"})
            )
        ])
        .round(3)
        .to_latex(
                    index = False,
                    caption = f"Índice de Grubel-Lloyd da indústria de aeronaves e espaçonaves",
                    decimal = ",",
                    column_format = ''.join(['p{{{}cm}}'.format(width) for width in [6]+[1.5 for i in range(2010, 2017, 2)]]),
                    na_rep='-',
                    label = f"tab:ex6-gl",
        )   
    )


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5252\3264386060.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.concat([
